In [8]:
import requests
import bs4
import pandas as pd
import numpy as np
from cjw_mysql import *
from tqdm import tqdm
from utils import *

In [9]:
db = Maria()
db.setMaria(db='ohlcv')
code_list = db.mariaShowData('stock_info')['code'].tolist()
data = {}


In [10]:
for code in tqdm(code_list):
    try:
        res = requests.get(f'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701')
        df = pd.read_html(res.text)[0]
        cols = list(df.columns)[1:]
        cols = [col.replace('/', '_') for col in cols]
        df.columns = ['지표'] + cols
        df['지표'] = [curr.replace('계산에 참여한 계정 펼치기', '') for curr in df['지표'].tolist()]
#         df = df.replace('안정성비율', np.nan)
#         df = df.replace('성장성비율', np.nan)
#         df = df.replace('수익성비율', np.nan)
#         df = df.replace('활동성비율', np.nan)
#         df = df.dropna().reset_index(drop=True)
    except:
        continue
    if df.shape[0] > 0:
        data[code] = df

100%|██████████████████████████████████████████████████████████████████████████████| 2459/2459 [16:17<00:00,  2.52it/s]


In [11]:
import time
res = {}
subject_list = data['005930']['지표'].tolist()

std_df = pd.DataFrame(data['005930'].where(data['005930'].지표=='매출액').dropna().iloc[0]).reset_index(drop=False).drop(0)
std_df.columns = ['period', '005930']

for subject in subject_list:
    print(subject)
    time.sleep(1)
    res[subject] = pd.DataFrame()
    res[subject]['period'] = std_df['period']
    for code in tqdm(list(data.keys())):
        try:
            curr_df = pd.DataFrame(data[code].where(data[code].지표==subject).dropna().iloc[0]).reset_index(drop=False).drop(0)
            curr_df.columns = ['period', code]
            res[subject] = pd.merge(res[subject], curr_df, on=['period'], how='outer').reset_index(drop=True)
        except:
            pass

매출액


100%|██████████████████████████████████████████████████████████████████████████████| 2456/2456 [03:54<00:00, 10.48it/s]


매출원가


100%|██████████████████████████████████████████████████████████████████████████████| 2456/2456 [03:22<00:00, 12.13it/s]


매출총이익


...

100%|██████████████████████████████████████████████████████████████████████████████| 2456/2456 [04:24<00:00,  9.28it/s]


지배주주순이익


100%|██████████████████████████████████████████████████████████████████████████████| 2456/2456 [02:40<00:00, 15.29it/s]


비지배주주순이익


100%|██████████████████████████████████████████████████████████████████████████████| 2456/2456 [00:57<00:00, 42.92it/s]


In [12]:
res['매출액']

,period,000020,000040,000050,000070,000080,000100,000120,000140,000150,...,900310,900340,950110,950130,950140,950160,950170,950180,950190,950200
0,2018_12,3066.0,366.0,3514.0,25635.0,18856.0,15188.0,92197.0,18767.0,174493.0,...,482.0,962.0,NaN,402.0,1188.0,38.0,NaN,NaN,808.0,209.0
1,2019_12,3072.0,1326.0,3439.0,24886.0,20351.0,14804.0,104151.0,20263.0,180416.0,...,509.0,1116.0,NaN,431.0,1333.0,30.0,NaN,NaN,970.0,201.0
2,2020_12,2721.0,1178.0,3292.0,24712.0,22563.0,16199.0,107811.0,22470.0,169693.0,...,737.0,1305.0,787.0,1218.0,1372.0,36.0,NaN,NaN,1130.0,190.0
3,2021_03,718.0,227.0,826.0,7007.0,5351.0,3790.0,26926.0,5324.0,45203.0,...,81.0,361.0,NaN,2257.0,338.0,6.0,NaN,NaN,249.0,59.0
4,전년동기,670.0,286.0,740.0,5973.0,5339.0,3133.0,25154.0,5315.0,43019.0,...,85.0,283.0,658.0,101.0,310.0,10.0,5592.0,373.0,270.0,48.0
5,전년동기(%),7.2,-20.5,11.6,17.3,0.2,21.0,7.0,0.2,5.1,...,-4.4,27.4,19.6,2136.3,8.9,-38.4,-96.8,-12.8,-8.0,22.9
6,2019_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for subject in subject_list:
    res[subject].iloc[:4].to_csv(f'C:/data/subject_{subject}.csv', index=False, encoding='cp949')

In [14]:
from utils import *

which = 'subject'
wish_list = ['매출액']
data = get_data(data, which, wish_list)

NameError: name 'get_data' is not defined

In [49]:
subject_list

['유동비율',
 '당좌비율',
 '부채비율',
 '유보율',
 '이자보상배율',
 '자기자본비율',
 '매출액증가율',
 '판매비와관리비증가율',
 '영업이익증가율',
 'EBITDA증가율',
 'EPS증가율',
 '매출총이익율',
 '세전계속사업이익률',
 '영업이익률',
 'EBITDA마진율',
 'ROA',
 'ROE',
 'ROIC',
 '총자산회전율',
 '총부채회전율',
 '총자본회전율']